# Fancy a Image Detector?

Don't have time to manually look through photos and decide what it is? We got you!

With this telegram bot - equipped with AI capabilities - you will be able to instantly classify images at lightning speed.

# AI for Image Classification

In [ ]:
!pip install tensorflow==2.13.0
!pip install tflite-support==0.4.4

In [ ]:
from tflite_support.task import vision
from tflite_support.task import core
from tflite_support.task import processor
import requests
import zipfile

"""
Returns a TFLite Image Classifier using the pretrained model for image classification
"""
def get_model():
  model_url = "https://storage.googleapis.com/download.tensorflow.org/models/tflite/mobilenet_v1_1.0_224_quant_and_labels.zip"
  r = requests.get(model_url)
  with open('mobilenet_v1_1.0_224_quant_and_labels.zip','wb') as f:
    f.write(r.content)

  with zipfile.ZipFile("mobilenet_v1_1.0_224_quant_and_labels.zip","r") as zip_ref:
    zip_ref.extractall("mobilenet_v1_1.0_224_quant_and_labels")

  with open("/content/mobilenet_v1_1.0_224_quant_and_labels/labels_mobilenet_quant_v1_224.txt", 'r') as f:
    labels = list(map(str.strip, f.readlines()))

  classifier = vision.ImageClassifier.create_from_file("/content/mobilenet_v1_1.0_224_quant_and_labels/mobilenet_v1_1.0_224_quant.tflite")
  return classifier, labels

"""
Using the given classifier, classify the given image to one of the items in the labels.
"""
def classify_image(classifier, labels, image_path):
  image = vision.TensorImage.create_from_file(image_path)
  classification_result = classifier.classify(image)
  max_score_category = sorted(classification_result.classifications[0].categories, reverse=True, key=lambda x: x.score)[0]

  return labels[max_score_category.index]

In [ ]:
classifier, labels = get_model()
classify_image(classifier, labels, 'sparrow.jpg')

# Creating Telegram Chatbot
First, search BotFather in Telegram, and request for a new bot by typing /newbot.

You will be prompted to give your bot a username, and you will get the bot token.

Here, we will create a telegram chatbot that classifies images that you send.

In [ ]:
!pip install python-telegram-bot

In [ ]:
from telegram import Update
from telegram.ext import filters, ApplicationBuilder, ContextTypes, MessageHandler
import nest_asyncio

async def image_classifier(update: Update, context: ContextTypes.DEFAULT_TYPE):
    image_attachment = await update.message.effective_attachment[-1].get_file()
    image_path = await image_attachment.download_to_drive()

    await context.bot.send_message(chat_id=update.effective_chat.id, text=classify_image(classifier, labels, str(image_path)))

if __name__ == '__main__':
  application = ApplicationBuilder().token('').build()

  image_handler = MessageHandler(filters.PHOTO & (~filters.COMMAND), image_classifier)
  application.add_handler(image_handler)

  nest_asyncio.apply()
  application.run_polling()